# variation of GPT model

In this notebook, we'll walk through the steps required to train a GPT model on the wine review dataset,we already downloaded the dataset from kaggle.Downloaded data in json format

# import the packages

In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import json
import re
import string
from IPython.display import display, HTML

import tensorflow as tf
from tensorflow.keras import layers, models, losses, callbacks

# set the parameter

In [2]:
VOCAB_SIZE = 10000
MAX_LEN = 80
EMBEDDING_DIM = 256
KEY_DIM = 256
N_HEADS = 2
FEED_FORWARD_DIM = 256
VALIDATION_SPLIT = 0.2
SEED = 42
LOAD_MODEL = False
BATCH_SIZE = 32
EPOCHS = 5

 # 1 load the dataset

In [3]:
# Load the full dataset of wine review in json format
with open("winemag-data-130k-v2.json") as json_data:
    wine_data = json.load(json_data)

In [4]:
def clean_text(text):
    """Basic text cleaning function"""
    if not isinstance(text, str):
        return ""
    # Remove special characters and extra whitespace
    text = re.sub(r'[^\w\s]', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text.lower()

# 2 perform data processing for the transformer

In [5]:
# Clean each component and rebuild the filtered_data list
cleaned_filtered_data = []
for x in wine_data:
    # Ensure that the required keys exist and are not None
    if all(key in x and x[key] is not None for key in ['country', 'province', 'variety', 'description']):
        country = clean_text(x['country'])
        province = clean_text(x['province'])
        variety = clean_text(x['variety'])
        description = clean_text(x['description'])

        # Create formatted string
        review_text = f"wine review : {country} : {province} : {variety} : {description}"
        cleaned_filtered_data.append(review_text)

# Replace the original filtered_data with the cleaned version
filtered_data = cleaned_filtered_data

# You can now check the number of wines again to ensure the same count
n_wines = len(filtered_data)
print(f"{n_wines} recipes loaded after cleaning and re-formatting")

# You can also check an example from the cleaned data
example = filtered_data[25]
print(example)

129907 recipes loaded after cleaning and re-formatting
wine review : us : california : pinot noir : oak and earth intermingle around robust aromas of wet forest floor in this vineyard designated pinot that hails from a high elevation site small in production it offers intense full bodied raspberry and blackberry steeped in smoky spice and smooth texture


In [6]:
# Count the recipes
n_wines = len(filtered_data)
print(f"{n_wines} recipes loaded")

129907 recipes loaded


In [7]:
example = filtered_data[25]
print(example)

wine review : us : california : pinot noir : oak and earth intermingle around robust aromas of wet forest floor in this vineyard designated pinot that hails from a high elevation site small in production it offers intense full bodied raspberry and blackberry steeped in smoky spice and smooth texture


In [8]:
# Clean each component
country = clean_text(x['country'])
province = clean_text(x['province'])
variety = clean_text(x['variety'])
description = clean_text(x['description'])

# Create formatted string
review_text = f"wine review : {country} : {province} : {variety} : {description}"
filtered_data.append(review_text)


3. Tokenize the data

In [9]:
# Pad the punctuation, to treat them as separate 'words'
def pad_punctuation(s):
    s = re.sub(f"([{string.punctuation}, '\n'])", r" \1 ", s)
    s = re.sub(" +", " ", s)
    return s


text_data = [pad_punctuation(x) for x in filtered_data]

In [10]:
# Display an example of a recipe
example_data = text_data[25]
example_data

'wine review : us : california : pinot noir : oak and earth intermingle around robust aromas of wet forest floor in this vineyard designated pinot that hails from a high elevation site small in production it offers intense full bodied raspberry and blackberry steeped in smoky spice and smooth texture'

# Convert to a Tensorflow Dataset after cleaning

In [11]:
#Convert to a Tensorflow Dataset
text_ds = (
    tf.data.Dataset.from_tensor_slices(text_data)
    .batch(BATCH_SIZE)
    .shuffle(1000)
)

In [12]:
# Create a vectorisation layer
vectorize_layer = layers.TextVectorization(
    standardize="lower",
    max_tokens=VOCAB_SIZE,
    output_mode="int",
    output_sequence_length=MAX_LEN + 1,
)

In [13]:
# Adapt the layer to the training set
vectorize_layer.adapt(text_ds)
vocab = vectorize_layer.get_vocabulary()

In [14]:
# Display some token:word mappings
for i, word in enumerate(vocab[:10]):
    print(f"{i}: {word}")

0: 
1: [UNK]
2: :
3: and
4: the
5: wine
6: a
7: of
8: review
9: with


In [15]:
# Display the same example converted to ints
example_tokenised = vectorize_layer(example_data)
print(example_tokenised.numpy())

[   5    8    2   16    2   25    2   39   58    2   50    3  237 4112
  448  626   22    7  489  492  659   14   10  137 2201   39   21 2470
   28    6  218 2192  942  587   14  983   12   71  230   60   77   92
    3   69 2616   14  194   44    3  120   72    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0]


# 4.  Create the Training Set

In [16]:
# Create the training set of recipes and the same text shifted by one word
def prepare_inputs(text):
    text = tf.expand_dims(text, -1)
    tokenized_sentences = vectorize_layer(text)
    x = tokenized_sentences[:, :-1]
    y = tokenized_sentences[:, 1:]
    return x, y


train_ds = text_ds.map(prepare_inputs)


In [17]:
example_input_output = train_ds.take(1).get_single_element()

In [18]:
example_input_output

(<tf.Tensor: shape=(32, 80), dtype=int64, numpy=
 array([[   5,    8,    2, ...,    0,    0,    0],
        [   5,    8,    2, ...,  101, 1932,    3],
        [   5,    8,    2, ...,    0,    0,    0],
        ...,
        [   5,    8,    2, ...,    0,    0,    0],
        [   5,    8,    2, ...,    0,    0,    0],
        [   5,    8,    2, ...,    0,    0,    0]])>,
 <tf.Tensor: shape=(32, 80), dtype=int64, numpy=
 array([[   8,    2,   16, ...,    0,    0,    0],
        [   8,    2,   38, ..., 1932,    3, 3120],
        [   8,    2,   36, ...,    0,    0,    0],
        ...,
        [   8,    2,  147, ...,    0,    0,    0],
        [   8,    2,  147, ...,    0,    0,    0],
        [   8,    2,   80, ...,    0,    0,    0]])>)

In [19]:
# Example Input
example_input_output[0][0]

<tf.Tensor: shape=(80,), dtype=int64, numpy=
array([   5,    8,    2,   16,    2,   25,    2,   41,   40,    2,  126,
        502,   10,   11,    4,  241, 9301,  920,   41,   14,  113,    3,
          6, 1125, 3357,  168,    4, 1378,   12,   78,   34,  344,   13,
          7,   69,    3,   33,   32,  494,  327,  207,    3,  194,  704,
        723,  138, 1809,  623,  183,   30,   31,   10, 1735,  168,    4,
        385,  824,  113,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0])>

In [20]:
# Example Output (shifted by one token)
example_input_output[1][0]

<tf.Tensor: shape=(80,), dtype=int64, numpy=
array([   8,    2,   16,    2,   25,    2,   41,   40,    2,  126,  502,
         10,   11,    4,  241, 9301,  920,   41,   14,  113,    3,    6,
       1125, 3357,  168,    4, 1378,   12,   78,   34,  344,   13,    7,
         69,    3,   33,   32,  494,  327,  207,    3,  194,  704,  723,
        138, 1809,  623,  183,   30,   31,   10, 1735,  168,    4,  385,
        824,  113,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0])>

# 5. Create the causal attention mask function

In [21]:
def causal_attention_mask(batch_size, n_dest, n_src, dtype):
    i = tf.range(n_dest)[:, None]
    j = tf.range(n_src)
    m = i >= j - n_src + n_dest
    mask = tf.cast(m, dtype)
    mask = tf.reshape(mask, [1, n_dest, n_src])
    mult = tf.concat(
        [tf.expand_dims(batch_size, -1), tf.constant([1, 1], dtype=tf.int32)], 0
    )
    return tf.tile(mask, mult)


np.transpose(causal_attention_mask(1, 10, 10, dtype=tf.int32)[0])

array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [0, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [0, 0, 1, 1, 1, 1, 1, 1, 1, 1],
       [0, 0, 0, 1, 1, 1, 1, 1, 1, 1],
       [0, 0, 0, 0, 1, 1, 1, 1, 1, 1],
       [0, 0, 0, 0, 0, 1, 1, 1, 1, 1],
       [0, 0, 0, 0, 0, 0, 1, 1, 1, 1],
       [0, 0, 0, 0, 0, 0, 0, 1, 1, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1]], dtype=int32)

# 6. Create a Transformer Block layer

In [22]:
class TransformerBlock(layers.Layer):
    def __init__(self, num_heads, key_dim, embed_dim, ff_dim, dropout_rate=0.1):
        super(TransformerBlock, self).__init__()
        self.num_heads = num_heads
        self.key_dim = key_dim
        self.embed_dim = embed_dim
        self.ff_dim = ff_dim
        self.dropout_rate = dropout_rate
        self.attn = layers.MultiHeadAttention(
            num_heads, key_dim, output_shape=embed_dim
        )
        self.dropout_1 = layers.Dropout(self.dropout_rate)
        self.ln_1 = layers.LayerNormalization(epsilon=1e-6)
        self.ffn_1 = layers.Dense(self.ff_dim, activation="relu")
        self.ffn_2 = layers.Dense(self.embed_dim)
        self.dropout_2 = layers.Dropout(self.dropout_rate)
        self.ln_2 = layers.LayerNormalization(epsilon=1e-6)

    def call(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size = input_shape[0]
        seq_len = input_shape[1]
        causal_mask = causal_attention_mask(
            batch_size, seq_len, seq_len, tf.bool
        )
        attention_output, attention_scores = self.attn(
            inputs,
            inputs,
            attention_mask=causal_mask,
            return_attention_scores=True,
        )
        attention_output = self.dropout_1(attention_output)
        out1 = self.ln_1(inputs + attention_output)
        ffn_1 = self.ffn_1(out1)
        ffn_2 = self.ffn_2(ffn_1)
        ffn_output = self.dropout_2(ffn_2)
        return (self.ln_2(out1 + ffn_output), attention_scores)

    def get_config(self):
        config = super().get_config()
        config.update(
            {
                "key_dim": self.key_dim,
                "embed_dim": self.embed_dim,
                "num_heads": self.num_heads,
                "ff_dim": self.ff_dim,
                "dropout_rate": self.dropout_rate,
            }
        )
        return config

# 7. Create the Token and Position Embedding

In [23]:
class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, max_len, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.max_len = max_len
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim
        self.token_emb = layers.Embedding(
            input_dim=vocab_size, output_dim=embed_dim
        )
        self.pos_emb = layers.Embedding(input_dim=max_len, output_dim=embed_dim)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions

    def get_config(self):
        config = super().get_config()
        config.update(
            {
                "max_len": self.max_len,
                "vocab_size": self.vocab_size,
                "embed_dim": self.embed_dim,
            }
        )
        return config

# 8. Build the Transformer model

In [24]:
inputs = layers.Input(shape=(None,), dtype=tf.int32)
x = TokenAndPositionEmbedding(MAX_LEN, VOCAB_SIZE, EMBEDDING_DIM)(inputs)
x, attention_scores = TransformerBlock(
    N_HEADS, KEY_DIM, EMBEDDING_DIM, FEED_FORWARD_DIM
)(x)
outputs = layers.Dense(VOCAB_SIZE, activation="softmax")(x)
gpt = models.Model(inputs=inputs, outputs=[outputs, attention_scores])
gpt.compile("adam", loss=[losses.SparseCategoricalCrossentropy(), None])

In [25]:
gpt.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, None)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ token_and_position_embedding    │ (None, None, 256)      │     2,580,480 │
│ (TokenAndPositionEmbedding)     │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_block               │ [(None, None, 256),    │       658,688 │
│ (TransformerBlock)              │ (None, 2, None, None)] │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, None, 10000)    │     2,570,000 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,809,168 (22.16 MB)

 Trainable params: 5,809,168 (22.16 MB)

 Non-trainable params: 0 (0.00 B)

In [26]:
if LOAD_MODEL:
    # model.load_weights('./models/model')
    gpt = models.load_model("./models/gpt", compile=True)

# 9. Train the Transformer

In [27]:
# Create a TextGenerator checkpoint
class TextGenerator(callbacks.Callback):
    def __init__(self, index_to_word, top_k=10):
        self.index_to_word = index_to_word
        self.word_to_index = {
            word: index for index, word in enumerate(index_to_word)
        }

    def sample_from(self, probs, temperature):
        probs = probs ** (1 / temperature)
        probs = probs / np.sum(probs)
        return np.random.choice(len(probs), p=probs), probs

    def generate(self, start_prompt, max_tokens, temperature):
        start_tokens = [
            self.word_to_index.get(x, 1) for x in start_prompt.split()
        ]
        sample_token = None
        info = []
        while len(start_tokens) < max_tokens and sample_token != 0:
            x = np.array([start_tokens])
            y, att = self.model.predict(x, verbose=0)
            sample_token, probs = self.sample_from(y[0][-1], temperature)
            info.append(
                {
                    "prompt": start_prompt,
                    "word_probs": probs,
                    "atts": att[0, :, -1, :],
                }
            )
            start_tokens.append(sample_token)
            start_prompt = start_prompt + " " + self.index_to_word[sample_token]
        print(f"\ngenerated text:\n{start_prompt}\n")
        return info

    def on_epoch_end(self, epoch, logs=None):
        self.generate("wine review", max_tokens=80, temperature=1.0)

In [28]:
# Create a model save checkpoint
model_checkpoint_callback = callbacks.ModelCheckpoint(
    filepath="./checkpoint/checkpoint.weights.h5",
    save_weights_only=True,
    save_freq="epoch",
    verbose=0,
)

tensorboard_callback = callbacks.TensorBoard(log_dir="./logs")

# Tokenize starting prompt
text_generator = TextGenerator(vocab)

# Compile and Train



In [29]:
gpt.fit(
    train_ds,
    epochs=EPOCHS,
    callbacks=[model_checkpoint_callback, tensorboard_callback, text_generator],
)

Epoch 1/5
4059/4060 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 2.5153
generated text:
wine review : italy : tuscany : red blend : made without the merlot merlot colorino and 40 nero this opens with notes of ripe black skinned fruit and a balsamic note the dense palate offers strawberry extract and mature black pepper chewy tannins 

4060/4060 ━━━━━━━━━━━━━━━━━━━━ 154s 36ms/step - loss: 2.5151
Epoch 2/5
4060/4060 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 1.9440
generated text:
wine review : us : california : seyval blanc : this wine is dry and vibrant in acidity as well as ripe apple mocha and white peach flavors it s a bit sweet but relatively light in body a wine that finishes slightly sweet with good acidity 

4060/4060 ━━━━━━━━━━━━━━━━━━━━ 93s 23ms/step - loss: 1.9439
Epoch 3/5
4058/4060 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 1.8665
generated text:
wine review : australia : south australia : chardonnay : sweet scents of vanilla and popcorn lead to a medium bodied chardonnay with s

In [32]:
# Save the final model
gpt.save("/content/sample_data/gpt.keras")

# Text Generation with Different Temperatures

In [35]:
def print_probs(info, vocab, top_k=5):
    for i in info:
        highlighted_text = []
        for word, att_score in zip(
            i["prompt"].split(), np.mean(i["atts"], axis=0)
        ):
            highlighted_text.append(
                '<span style="background-color:rgba(135,206,250,'
                + str(att_score / max(np.mean(i["atts"], axis=0)))
                + ');">'
                + word
                + "</span>"
            )
        highlighted_text = " ".join(highlighted_text)
        display(HTML(highlighted_text))

        word_probs = i["word_probs"]
        p_sorted = np.sort(word_probs)[::-1][:top_k]
        i_sorted = np.argsort(word_probs)[::-1][:top_k]
        for p, i in zip(p_sorted, i_sorted):
            print(f"{vocab[i]}:   \t{np.round(100*p,2)}%")
        print("--------\n")

In [36]:
info = text_generator.generate(
    "wine review : us", max_tokens=80, temperature=1.0
)


generated text:
wine review : us : california : zinfandel : full bodied and lushly textured this wine has briary black pepper and black berry flavors with smoky wood [UNK] fully generous texture it has a deep inky dark color it is full and a subtle tannic full bodied wine 



In [38]:
info = text_generator.generate(
    "wine review : italy", max_tokens=80, temperature=0.5
)


generated text:
wine review : italy : tuscany : sangiovese : this riserva has a clean and correct nose with bright berry aromas followed by black cherry leather and tobacco flavors it s a very nice wine to pair with pasta or meat 



In [39]:
info = text_generator.generate(
    "wine review : germany", max_tokens=80, temperature=0.5
)
print_probs(info, vocab)


generated text:
wine review : germany : mosel : riesling : fresh lime and lemon flavors are accented by a touch of saffron in this off dry riesling it s slightly austere and light in style yet fresh with a clean refreshing finish 



::   	100.0%
saar:   	0.0%
cabernet:   	0.0%
grosso:   	0.0%
merlot:   	0.0%
--------



mosel:   	60.59000015258789%
rheingau:   	25.0%
rheinhessen:   	7.909999847412109%
pfalz:   	3.700000047683716%
nahe:   	1.4299999475479126%
--------



::   	99.27999877929688%
saar:   	0.7200000286102295%
donauland:   	0.0%
central:   	0.0%
spain:   	0.0%
--------



riesling:   	99.9800033569336%
white:   	0.009999999776482582%
pinot:   	0.009999999776482582%
rosé:   	0.0%
gewürztraminer:   	0.0%
--------



...

a:   	96.18000030517578%
an:   	1.2200000286102295%
crisp:   	0.4699999988079071%
zesty:   	0.23999999463558197%
juicy:   	0.20999999344348907%
--------



long:   	16.690000534057617%
lingering:   	14.029999732971191%
brisk:   	11.65999984741211%
clean:   	9.239999771118164%
hint:   	7.03000020980835%
--------



finish:   	43.11000061035156%
brisk:   	17.309999465942383%
refreshing:   	7.900000095367432%
crisp:   	7.53000020980835%
minerally:   	5.440000057220459%
--------



finish:   	98.76000213623047%
minerality:   	0.5099999904632568%
acidity:   	0.2199999988079071%
mouthfeel:   	0.07999999821186066%
streak:   	0.07000000029802322%
--------



:   	99.36000061035156%
drink:   	0.6000000238418579%
it:   	0.009999999776482582%
that:   	0.009999999776482582%
a:   	0.0%
--------

